In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as pyplot
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

In [2]:
andres_andres = pd.read_csv('grabaciones/andres/andres.csv', index_col=0)
andres_abajo = pd.read_csv('grabaciones/andres/abajo.csv', index_col=0)
andres_arriba = pd.read_csv('grabaciones/andres/arriba.csv', index_col=0)
andres_izquierda = pd.read_csv('grabaciones/andres/izquierda.csv', index_col=0)
andres_derecha = pd.read_csv('grabaciones/andres/derecha.csv', index_col=0)
vanessa_vanessa = pd.read_csv('grabaciones/vanessa/vanessa.csv', index_col=0)
vanessa_abajo = pd.read_csv('grabaciones/vanessa/abajo.csv', index_col=0)
vanessa_arriba = pd.read_csv('grabaciones/vanessa/arriba.csv', index_col=0)
vanessa_izquierda = pd.read_csv('grabaciones/vanessa/izquierda.csv', index_col=0)
vanessa_derecha = pd.read_csv('grabaciones/vanessa/derecha.csv', index_col=0)

pd.set_option('precision', 1)
numero_estimadores = 500

## Clasificaciones
- Nombres: Clasificar entre Vanessa o Andrés.
- Palabras de Andrés: Clasificar entre arriba, abajo, derecha e izquierda.
- Palabras de Vanessa: Clasificar entre arriba, abajo, derecha e izquierda.

## Clasificador completo
Combina todos los dataframes en uno solo.

In [3]:
datos_todos = pd.concat([andres_andres, andres_abajo, andres_arriba, andres_derecha, andres_izquierda,
                        vanessa_vanessa, vanessa_arriba, vanessa_abajo, vanessa_izquierda, vanessa_derecha]
                        , ignore_index=True)
len(datos_todos)

970

In [4]:
medias_totales = datos_todos.groupby(datos_todos['Palabra']).mean()
std_totales = datos_todos.groupby(datos_todos['Palabra']).std()
print("Medias de todos los datos:\n")
print(medias_totales)
print("\n\nDesviación estándar de todos los datos: \n")
print(std_totales)

Medias de todos los datos:

                   20-100 Hz  100-200 Hz  200-300 Hz  300-400 Hz  400-500 Hz  \
Palabra                                                                        
andres/abajo         16622.4     42052.3     19694.2     13061.9     19030.1   
andres/andres        16545.0     34731.4     31022.9     24550.1     40597.8   
andres/arriba        16899.1     47359.2     45605.7     32183.3     58067.5   
andres/derecha       17581.2     42470.6     36733.4     31722.4     57158.2   
andres/izquierda     15513.1     40574.0     48361.3     42433.5     67490.3   
vanessa/abajo        28808.8     33061.1     39492.9     18790.7     29096.0   
vanessa/arriba       36651.8     42662.8     39300.5     47940.0     49649.2   
vanessa/derecha      33914.9     47303.8     33812.0     59254.0     36767.9   
vanessa/izquierda    29994.1     42722.3     40637.6     37958.0     39732.0   
vanessa/vanessa      31497.3     48024.0     34486.6     25880.7     25385.5   

           

### Poniendo a prueba el clasificador

In [5]:
datos_todos_x = datos_todos.iloc[:, :-1]
datos_todos_y = datos_todos['Palabra']

le = LabelEncoder()
le.fit(datos_todos_y)
#datos_todos_y = le.transform(datos_todos_y)

x_train, x_test, y_train, y_test = train_test_split(datos_todos_x, datos_todos_y)

In [6]:
#clf_lr = LinearRegression()
#clf_lr.fit(x_train, y_train)
#predicted = clf_lr.predict(x_test).astype(int)

#clf_svc = SVC()
#clf_svc.fit(x_train, y_train)
#predicted = clf_svc.predict(x_test)

#clf_dt = DecisionTreeClassifier()
#clf_dt.fit(x_train, y_train)
#predicted = clf_dt.predict(x_test)

clf_rf = RandomForestClassifier(n_estimators=numero_estimadores)
clf_rf.fit(x_train, y_train)
predicted = clf_rf.predict(x_test)

In [7]:
datos_todos_x_escalados = preprocessing.scale(datos_todos_x)

clf_rf = RandomForestClassifier(n_estimators=numero_estimadores)
scores = cross_val_score(clf_rf, datos_todos_x_escalados, datos_todos_y, cv=10)
scores2 = cross_val_score(clf_rf, datos_todos_x, datos_todos_y, cv=10)
print("Precision: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores2.mean(), scores2.std() * 2))

Precision: 0.94 (+/- 0.09)
Precision: 0.94 (+/- 0.07)


In [8]:
print(classification_report(y_test, predicted))

                   precision    recall  f1-score   support

     andres/abajo       1.00      1.00      1.00        23
    andres/andres       0.87      1.00      0.93        13
    andres/arriba       1.00      1.00      1.00        22
   andres/derecha       0.86      0.93      0.89        27
 andres/izquierda       0.96      0.84      0.90        31
    vanessa/abajo       1.00      1.00      1.00        31
   vanessa/arriba       0.92      0.92      0.92        26
  vanessa/derecha       0.88      0.92      0.90        25
vanessa/izquierda       0.96      0.89      0.93        28
  vanessa/vanessa       0.94      1.00      0.97        17

      avg / total       0.94      0.94      0.94       243



In [9]:
print(confusion_matrix(y_test, predicted))

[[23  0  0  0  0  0  0  0  0  0]
 [ 0 13  0  0  0  0  0  0  0  0]
 [ 0  0 22  0  0  0  0  0  0  0]
 [ 0  1  0 25  1  0  0  0  0  0]
 [ 0  0  0  4 26  0  0  1  0  0]
 [ 0  0  0  0  0 31  0  0  0  0]
 [ 0  1  0  0  0  0 24  0  0  1]
 [ 0  0  0  0  0  0  1 23  1  0]
 [ 0  0  0  0  0  0  1  2 25  0]
 [ 0  0  0  0  0  0  0  0  0 17]]


## Clasificador nombres
Combina los dataframes de los nombres en uno solo

In [10]:
datos_nombres = pd.concat([andres_andres, vanessa_vanessa], ignore_index=True)

In [11]:
medias_nombres = datos_nombres.groupby(datos_nombres['Palabra']).mean()
std_nombres = datos_nombres.groupby(datos_nombres['Palabra']).std()
print("Media de los nombres:\n")
print(medias_nombres)
print("\n\nDesviación estándar de los nombres:\n")
print(std_nombres)

Media de los nombres:

                 20-100 Hz  100-200 Hz  200-300 Hz  300-400 Hz  400-500 Hz  \
Palabra                                                                      
andres/andres      16545.0     34731.4     31022.9     24550.1     40597.8   
vanessa/vanessa    31497.3     48024.0     34486.6     25880.7     25385.5   

                 500-600 Hz  600-700 Hz  700-800 Hz  800-900 Hz  900-1000 Hz  \
Palabra                                                                        
andres/andres       22006.1     10078.5      8010.4      7152.5       6749.2   
vanessa/vanessa     38775.5     42767.0     52385.6     24895.9      11029.1   

                 1000-1100 Hz  1100-1200 Hz  1200-1300 Hz  1300-1400 Hz  \
Palabra                                                                   
andres/andres          3868.3        3078.0        3638.6        3727.3   
vanessa/vanessa        7799.7        8198.7        6692.6        8790.2   

                 1400-1500 Hz  1500-1600 H

### Poniendo a prueba el clasificador

In [12]:
datos_nombres_x = datos_nombres.iloc[:, :-1]
datos_nombres_y = datos_nombres['Palabra']

le = LabelEncoder()
le.fit(datos_nombres_y)
#datos_nombres_y = le.transform(datos_nombres_y)

x_train, x_test, y_train, y_test = train_test_split(datos_nombres_x, datos_nombres_y)

In [13]:
#clf_lr = LinearRegression()
#clf_lr.fit(x_train, y_train)
#predicted = clf_lr.predict(x_test).astype(int)

#clf_svc = SVC()
#clf_svc.fit(x_train, y_train)
#predicted = clf_svc.predict(x_test)

#clf_dt = DecisionTreeClassifier()
#clf_dt.fit(x_train, y_train)
#predicted = clf_dt.predict(x_test)

clf_rf = RandomForestClassifier(n_estimators=numero_estimadores)
clf_rf.fit(x_train, y_train)
predicted = clf_rf.predict(x_test)

In [14]:
clf_rf = RandomForestClassifier(n_estimators=numero_estimadores)
scores = cross_val_score(clf_rf, datos_nombres_x, datos_nombres_y, cv=5)
print("Precision: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Precision: 0.91 (+/- 0.30)


In [15]:
print(classification_report(y_test, predicted))

                 precision    recall  f1-score   support

  andres/andres       0.96      1.00      0.98        22
vanessa/vanessa       1.00      0.95      0.98        21

    avg / total       0.98      0.98      0.98        43



In [16]:
print(confusion_matrix(y_test, predicted))

[[22  0]
 [ 1 20]]


## Clasificador palabras Andrés
Combina todos los dataframes que contienen palabras dichas por Andrés.

In [17]:
datos_andres = pd.concat([andres_abajo, andres_arriba, andres_izquierda, andres_derecha], ignore_index=True)
print(datos_andres.groupby(datos_andres['Palabra']).describe())

                 100-200 Hz                                              \
                      count     mean     std      min      25%      50%   
Palabra                                                                   
andres/abajo          100.0  42052.3  8892.6  23506.1  34860.5  41037.5   
andres/arriba         100.0  47359.2  5926.4  34885.2  42958.0  47338.2   
andres/derecha        100.0  42470.6  9055.0  25757.9  36569.0  41553.7   
andres/izquierda      100.0  40574.0  7018.6  23789.5  36271.0  40926.6   

                                   1000-1100 Hz            ...    800-900 Hz  \
                      75%      max        count     mean   ...           75%   
Palabra                                                    ...                 
andres/abajo      48057.2  65863.3        100.0  24252.4   ...       18176.3   
andres/arriba     51945.5  62062.8        100.0   5719.9   ...        7062.3   
andres/derecha    46394.3  77259.1        100.0   2688.0   ...        5797

In [18]:
medias_andres = datos_andres.groupby(datos_andres['Palabra']).mean()
std_andres = datos_andres.groupby(datos_andres['Palabra']).std()
print("Media de las palabras dichas por Andrés:\n")
print(medias_andres)
print("\n\nDesviación estándar de las palabras dichas por Andrés:\n")
print(std_andres)

Media de las palabras dichas por Andrés:

                  20-100 Hz  100-200 Hz  200-300 Hz  300-400 Hz  400-500 Hz  \
Palabra                                                                       
andres/abajo        16622.4     42052.3     19694.2     13061.9     19030.1   
andres/arriba       16899.1     47359.2     45605.7     32183.3     58067.5   
andres/derecha      17581.2     42470.6     36733.4     31722.4     57158.2   
andres/izquierda    15513.1     40574.0     48361.3     42433.5     67490.3   

                  500-600 Hz  600-700 Hz  700-800 Hz  800-900 Hz  900-1000 Hz  \
Palabra                                                                         
andres/abajo         26367.7     33897.1     38115.3     16348.2      25500.6   
andres/arriba        41304.8     28714.9     15609.4      6474.5       4958.8   
andres/derecha       35041.8     15637.7      6736.7      5003.4       2930.3   
andres/izquierda     46297.1     16496.8      7499.9      5060.6       3543.1 

### Poniendo a prueba el clasificador

In [19]:
datos_andres_x = datos_andres.iloc[:, :-1]
datos_andres_y = datos_andres['Palabra']

le = LabelEncoder()
le.fit(datos_andres_y)
#datos_nombres_y = le.transform(datos_nombres_y)

x_train, x_test, y_train, y_test = train_test_split(datos_andres_x, datos_andres_y)

In [20]:
#clf_lr = LinearRegression()
#clf_lr.fit(x_train, y_train)
#predicted = clf_lr.predict(x_test).astype(int)

#clf_svc = SVC()
#clf_svc.fit(x_train, y_train)
#predicted = clf_svc.predict(x_test)

#clf_dt = DecisionTreeClassifier()
#clf_dt.fit(x_train, y_train)
#predicted = clf_dt.predict(x_test)

clf_rf = RandomForestClassifier(n_estimators=numero_estimadores)
clf_rf.fit(x_train, y_train)
predicted = clf_rf.predict(x_test)

In [21]:
clf_rf = RandomForestClassifier(n_estimators=numero_estimadores)
scores = cross_val_score(clf_rf, datos_andres_x, datos_andres_y, cv=10)
print("Precision: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Precision: 0.98 (+/- 0.14)


In [22]:
print(classification_report(y_test, predicted))

                  precision    recall  f1-score   support

    andres/abajo       1.00      1.00      1.00        29
   andres/arriba       1.00      1.00      1.00        26
  andres/derecha       0.94      1.00      0.97        29
andres/izquierda       1.00      0.88      0.93        16

     avg / total       0.98      0.98      0.98       100



In [23]:
print(confusion_matrix(y_test, predicted))

[[29  0  0  0]
 [ 0 26  0  0]
 [ 0  0 29  0]
 [ 0  0  2 14]]


## Clasificador palabras Vanessa
Combina todos los dataframes que contienen palabras dichas por Vanessa.

In [24]:
datos_vanessa = pd.concat([vanessa_abajo, vanessa_arriba, vanessa_izquierda, vanessa_derecha], ignore_index=True)

In [25]:
medias_vanessa = datos_vanessa.groupby(datos_vanessa['Palabra']).mean()
std_vanessa = datos_vanessa.groupby(datos_vanessa['Palabra']).std()
print("Media de las palabras dichas por Vanessa:\n")
print(medias_vanessa)
print("\n\nDesviación estándar de las palabras dichas por Vanessa:\n")
print(std_vanessa)

Media de las palabras dichas por Vanessa:

                   20-100 Hz  100-200 Hz  200-300 Hz  300-400 Hz  400-500 Hz  \
Palabra                                                                        
vanessa/abajo        28808.8     33061.1     39492.9     18790.7     29096.0   
vanessa/arriba       36651.8     42662.8     39300.5     47940.0     49649.2   
vanessa/derecha      33914.9     47303.8     33812.0     59254.0     36767.9   
vanessa/izquierda    29994.1     42722.3     40637.6     37958.0     39732.0   

                   500-600 Hz  600-700 Hz  700-800 Hz  800-900 Hz  \
Palabra                                                             
vanessa/abajo         32697.6     54258.0     58170.8     80961.3   
vanessa/arriba        40787.5     39929.0     29149.3     29337.4   
vanessa/derecha       46403.8     17719.7     12334.9      9985.4   
vanessa/izquierda     52615.7     21005.4     13306.3     10238.8   

                   900-1000 Hz  1000-1100 Hz  1100-1200 Hz  1

In [26]:
datos_vanessa_x = datos_vanessa.iloc[:, :-1]
datos_vanessa_y = datos_vanessa['Palabra']

le = LabelEncoder()
le.fit(datos_vanessa_y)
#datos_nombres_y = le.transform(datos_nombres_y)

x_train, x_test, y_train, y_test = train_test_split(datos_vanessa_x, datos_vanessa_y)

In [27]:
#clf_lr = LinearRegression()
#clf_lr.fit(x_train, y_train)
#predicted = clf_lr.predict(x_test).astype(int)

#clf_svc = SVC()
#clf_svc.fit(x_train, y_train)
#predicted = clf_svc.predict(x_test)

#clf_dt = DecisionTreeClassifier()
#clf_dt.fit(x_train, y_train)
#predicted = clf_dt.predict(x_test)

clf_rf = RandomForestClassifier(n_estimators=numero_estimadores)
clf_rf.fit(x_train, y_train)
predicted = clf_rf.predict(x_test)

In [28]:
clf_rf = RandomForestClassifier(n_estimators=numero_estimadores)
scores = cross_val_score(clf_rf, datos_vanessa_x, datos_vanessa_y, cv=5)
print("Precision: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Precision: 0.89 (+/- 0.13)


In [29]:
print(classification_report(y_test, predicted))

                   precision    recall  f1-score   support

    vanessa/abajo       1.00      1.00      1.00        31
   vanessa/arriba       0.92      1.00      0.96        23
  vanessa/derecha       1.00      0.79      0.88        28
vanessa/izquierda       0.77      0.94      0.85        18

      avg / total       0.94      0.93      0.93       100



In [30]:
print(confusion_matrix(y_test, predicted))

[[31  0  0  0]
 [ 0 23  0  0]
 [ 0  1 22  5]
 [ 0  1  0 17]]


# Guardado de los clasificadores

In [31]:
clf_rf_todos = RandomForestClassifier(n_estimators=2000, n_jobs=-1)
clf_rf_nombres = RandomForestClassifier(n_estimators=2000, n_jobs=-1)
clf_rf_andres = RandomForestClassifier(n_estimators=2000, n_jobs=-1)
clf_rf_vanessa = RandomForestClassifier(n_estimators=2000, n_jobs=-1)

In [32]:
clf_rf_todos.fit(datos_todos_x, datos_todos_y)
clf_rf_nombres.fit(datos_nombres_x, datos_nombres_y)
clf_rf_andres.fit(datos_andres_x, datos_andres_y)
clf_rf_vanessa.fit(datos_vanessa_x, datos_vanessa_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [33]:
with open('grabaciones/clasificadores/clasificador_completo.clf', 'wb') as archivo:
    pickle.dump(clf_rf_todos, archivo)
with open('grabaciones/clasificadores/clasificador_nombres.clf', 'wb') as archivo:
    pickle.dump(clf_rf_nombres, archivo)
with open('grabaciones/clasificadores/clasificador_andres.clf', 'wb') as archivo:
    pickle.dump(clf_rf_andres, archivo)
with open('grabaciones/clasificadores/clasificador_vanessa.clf', 'wb') as archivo:
    pickle.dump(clf_rf_vanessa, archivo)